<a href="https://colab.research.google.com/github/it21813702/ctse_ml_chatbot/blob/main/CTSE_ML_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Necessary Libraries

In [3]:
!pip install -q sentence-transformers                                   # Install sentence-transformers to generate text embeddings
!pip install -q faiss-cpu                                                         # Install FAISS for efficient vector similarity search
!pip install -q langchain                                                       # Install LangChain for chaining LLM + retriever components
!pip install -q transformers                                                  # Install HuggingFace Transformers (optional, if using model pipelines)


# Import Required Libraries

# for embedding lecture notes using MiniLM
from sentence_transformers import SentenceTransformer

# fo storing and searching text embeddings
import faiss
import numpy as np

# fo reading and processing text
import os
import re

print("✅ All libraries installed and imported successfully!")


✅ All libraries installed and imported successfully!


# Load and Preprocess Lecture Notes

In [5]:
!pip install -q pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 13.1 MB/s eta 0:00:00


In [29]:
# unzip lecture notes archive  into a folder
#          path in collab where folder is              place to extract to
!unzip -q "/content/CTSE_Lecture_Notes.zip" -d "/content/CTSE_Lecture_Notes"
print("✅ Files extracted to /content/CTSE_Lecture_Notes")



# list al files in the extracted folder
import os

# define the path where notes are located
notes_dir = "/content/CTSE_Lecture_Notes"

# print all files found in the folder
print("\n\n📄 Files found in lecture notes folder:")
print(os.listdir(notes_dir))



# import Required Libraries for PDF and PPTX
# Install PyPDFLoader (community extension for LangChain PDF reading)
!pip install -q langchain-community
!pip install -q python-pptx

# Import LangChain PDF loader and schema types
from langchain_community.document_loaders import PyPDFLoader
from pptx import Presentation
from langchain.schema import Document


# deefine helper Function to Load PPTX Files

# function reads all text from a .pptx (PowerPoint) file and returns it as a LangChain Document
def load_pptx_as_document(path):
    prs = Presentation(path)
    text_runs = []

    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                text_runs.append(shape.text)


    # combne all extracted text and wrap as a Document with metadata
    full_text = "\n".join(text_runs)
    return Document(page_content=full_text, metadata={"source": os.path.basename(path)})



# Load All Documents into LangChain Format

documents = []  # List to hold all documents


# loop through each file in the lecture notes folder
for file in os.listdir(notes_dir):
    path = os.path.join(notes_dir, file)

    # If it's a PDF, use the LangChain PDF loader (returns one doc per page)
    if file.endswith(".pdf"):
        loader = PyPDFLoader(path)
        pdf_docs = loader.load()
        documents.extend(pdf_docs)

    # If it's a PPTX, use the helper function
    elif file.endswith(".pptx"):
        ppt_doc = load_pptx_as_document(path)
        documents.append(ppt_doc)


# confirm and Preview Loaded Documents
print(f"\n\n✅ Total documents loaded: {len(documents)}")
print("\n\n📄 Sample content from the first document:\n")
print(documents[0].page_content[:500])


replace /content/CTSE_Lecture_Notes/ML Lec 2 - Part 2 LLM.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
✅ Files extracted to /content/CTSE_Lecture_Notes
📄 Files found in lecture notes folder:
['ML Lec 2 - Part 2 LLM.pdf', 'ML Lec 2 - Part 1.pdf']
✅ Total documents loaded: 60


📄 Sample content from the first document:

LLM Development Toolkit
Lecture 02 - Part 2 (Upgraded)
April 21, 2025
Lecture 02 - Part 2 (Upgraded) LLM Development Toolkit April 21, 2025 1 / 23


# Chunking and Embedding

In [7]:
# Split Documents into Chunks

# Splits long documents into smaller overlapping chunks
# So the model can read and reason over short, coherent text

from langchain.text_splitter import RecursiveCharacterTextSplitter



# this splitter breaks long documents into smaller parts while trying to respect sentence boundaries.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,     # max characters per chunk
    chunk_overlap=100   # overlap to preserve context between chunks

)


# Split the full list of loaded documents (from Step 2) into chunks
split_documents = text_splitter.split_documents(documents)



# confirm how many chunks were created
print(f"✅ Total chunks created: {len(split_documents)}")
print("📄 Example chunk:\n")
print(split_documents[0].page_content[:500])  # Preview a chunk


✅ Total chunks created: 69
📄 Example chunk:

LLM Development Toolkit
Lecture 02 - Part 2 (Upgraded)
April 21, 2025
Lecture 02 - Part 2 (Upgraded) LLM Development Toolkit April 21, 2025 1 / 23


In [8]:
# Uses all-MiniLM-L6-v2 to turn each chunk into a vector
# Embeddings allow fast, accurate semantic search

from langchain_community.embeddings import HuggingFaceEmbeddings


# Load a lightweight, Colab-compatible embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


print("\n\n✅ Embedding model loaded successfully.")

<ipython-input-8-0509adc2d73f>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embedding model loaded successfully.


In [9]:
# Stores all vectors in a FAISS index
# FAISS helps us quickly retrieve the most relevant chunks later

# 💾 Step 3.3: Store Embeddings in FAISS Vector Database

from langchain_community.vectorstores import FAISS

# Create a FAISS index from the embedded chunks
vectorstore = FAISS.from_documents(split_documents, embedding_model)

print("✅ FAISS vectorstore created and ready for retrieval.")

✅ FAISS vectorstore created and ready for retrieval.


# Build Chatbot with RetrievalQA and flan-t5-large

In [15]:
!pip install -q transformers

# 🤖 Step 4.2: Load flan-t5-large as a QA model using HuggingFace Transformers

from transformers import pipeline

# Use HuggingFace's pipeline to load flan-t5-large for text generation
# task="text2text-generation" works best with T5 models
qa_pipeline = pipeline(
    task="text2text-generation",
    model="google/flan-t5-large",
    device=0,
    max_new_tokens=200,  # give it room to finish
    temperature=0.3       # more deterministic
)

print("✅ flan-t5-large model loaded and ready.")

Device set to use cpu


✅ flan-t5-large model loaded and ready.


In [17]:
# create a Retriever from FAISS Vectorstore


# Use the vectorstore from Step 3 to create a retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

print("✅ Retriever ready to find relevant chunks.")

✅ Retriever ready to find relevant chunks.


In [24]:
# set Up langchain RetrievalQA Chain

from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate

# Wrap the HuggingFace pipeline into a LangChain-compatible LLM
llm = HuggingFacePipeline(pipeline=qa_pipeline)


# can define a prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
     You are a helpful AI tutor explaining machine learning concepts using lecture notes.
     Use the context below to fully and clearly answer the student's question.

     Context:
     {context}

     Question:
     {question}

      Answer:"""
)



# build the RetrievalQA chain using the retriever, LLM, and prompt
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",  # simplest format — stuff all retrieved docs into the prompt
    chain_type_kwargs={"prompt": prompt_template},
    return_source_documents=True  # useful if you want to display which notes were used
)



print("✅ RetrievalQA chain set up and ready for questions!")

✅ RetrievalQA chain set up and ready for questions!


In [25]:
# TESTING#######


# Ask a Question
# REPLACE with any question related to your CTSE lecture notes
user_question = "What is a GAN?"

# Run QA chain
response = qa_chain({"query": user_question})


# Display the final answer
print("📌 Question:", user_question)
print("💬 Answer:", response["result"])


# Optional--- Display  source documents that were used
print("\n📚 Source Document Excerpt:")
for i, doc in enumerate(response["source_documents"]):
    print(f"--- Document {i+1} ---")
    print(doc.page_content[:300], "\n")

📌 Question: What is a GAN?
💬 Answer: GAN is a type of neural network that is used for generative tasks, such as image or text generation.

📚 Source Document Excerpt:
--- Document 1 ---
SE4010 | Current Trends in SE| Introduction to Artificial Neural Networks| Jeewaka Perera
Generative Adversarial Neural Networks
•GAN is a type of neural network that is used for 
generative tasks, such as image or text generation.
•GAN consists of two neural networks: a generator 
network and a dis 

--- Document 2 ---
SE4010 | Current Trends in SE| Introduction to Artificial Neural Networks| Jeewaka Perera
Applications of GAN
•GAN has been used for a variety of generative tasks, 
including image generation, text generation, and 
music generation.
•GAN can also be used for data augmentation, where it 
is used to g 

--- Document 3 ---
signal, which is then passed on to the next layer.
•MLP is a feedforward network, which means that 
information flows in only one direction, from the 
input layer to the o

# Build Chatbot UI Using Gradio

In [26]:
!pip install -q gradio
import gradio as gr


# define the Chatbot Logic Function



# This function will be called every time the user submits a question
def chatbot_response(question):
    if not question.strip():
        return "Please enter a question."

    # Query the QA chain using your retriever and LLM
    response = qa_chain({"query": question})

    # Extract the generated answer
    answer = response["result"]

    # Optionally extract source documents
    sources = ""
    for i, doc in enumerate(response["source_documents"]):
        sources += f"\n\n📄 Source {i+1}:\n{doc.page_content[:300]}..."

    return f"🧠 **Answer:**\n{answer}\n\n---\n🗂️ **Sources Used:**{sources}"


In [27]:
# build the Gradio UI Layout

# Create a textbox input, output text area, and a submit button
iface = gr.Interface(
    fn=chatbot_response,                     # function to call

    inputs=gr.Textbox(
        lines=2,
        placeholder="Ask a question about your CTSE lecture notes...",
        label="Your Question"
    ),


    outputs=gr.Markdown(),                   # markdown to format answer nicely

    title="📚 CTSE Lecture Notes Chatbot",

    description="Ask questions about lecture notes (ML, LLMs, ANN) and get helpful answers!",

    allow_flagging="never"                   # Disable user flagging for simplicity
)


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


In [28]:
# Launch the Interface

iface.launch(share=True)                                     # share=True gives a public link

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d95325ae193190c6e8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
